#####  Inputs:  
######  --------            Step1_DataPreparation/D2_CSV_data/prediction_train.csv
######  --------            Step2_MyNet/net

#####  Outputs: 
######  --------            Outputs: prediction.csv

In [1]:
cd /Users/Nevaeh/PythonCSCourse/XcodePython/FindCell2/Step2_MyNet

/Users/Nevaeh/PythonCSCourse/XcodePython/FindCell2/Step2_MyNet


In [17]:
import tensorflow as tf
import pandas as pd
import numpy as np
import RHX_Input
import RHX_Network
import RHX_Transformation

tf.reset_default_graph() #This will ensure that the variables get the names you intended, but it will invalidate previously-created graphs



###--------------------------------------------------------------###
'''            --     Part 1: Loading  images    --              '''

My_Data = RHX_Input.LoadImage(directory='../Step1_DataPreparation/D2_CSV_data',Filename='prediction',LabelSize=1) # Get location
My_Data.RandomLoad()  # Load data

InputSize  = My_Data.InputSize  # Get InputSize
OutputSize = My_Data.OutputSize # Get OutputSize

###---------------------------------------------------------------###


###---------------------------------------------------------------###
'''            --     Part 2: define  placeholder    --           '''

with tf.name_scope('inputs'):
    xs          = tf.placeholder(tf.float32, [None,InputSize*InputSize],   name = 'x_input')
    ys          = tf.placeholder(tf.float32, [None,OutputSize*OutputSize], name = 'y_input')

    keep_prob   = tf.placeholder(tf.float32, name ='Dropout')
    
    tst         = tf.placeholder(tf.bool)    # test flag for batch norm
    iter        = tf.placeholder(tf.int32)

    is_training = tf.placeholder(tf.float32, name='is_training')

###-- -------------------------------------------------- --###



###---------------------------------------------------------------###
'''     --     Part 3:  get prediction from network    --         '''

IMAGE_x_SIZE = InputSize
IMAGE_y_SIZE = OutputSize

#x_image,y_image = RHX_Transformation.inputs(xs,ys,InputSize,OutputSize,IMAGE_x_SIZE,IMAGE_y_SIZE)
x_image,y_image = RHX_Transformation.distorted_inputs(xs,ys,InputSize,OutputSize,IMAGE_x_SIZE,IMAGE_y_SIZE)


prediction = RHX_Network.inference(x_image,IMAGE_x_SIZE)


# ###-- ---------------------------------------- --###
# ###-- Part 4: load parameters from nets: start --###

sess   = tf.Session()


saver = tf.train.Saver()
saver.restore(sess,"my_net/save_net.ckpt")


###-- Part 4: load parameters from nets: start  --###
###-- -------------------------------------- --###


print "Done!"



# Combine Prediction and Input
Output =  pd.DataFrame([])

for i in np.arange(My_Data.Frames):
    x_data_test, y_data_test = My_Data.Batch_Prediction(i)
    Predict_data,X_flip,Y_flip = sess.run([prediction,x_image,y_image], feed_dict={xs: x_data_test, ys: y_data_test, keep_prob: 1,is_training: 1,tst: False,iter: 1})

    Temp = pd.DataFrame(np.concatenate([Predict_data.reshape([-1]),X_flip.reshape([-1])]))
    Output = pd.concat([Output,Temp.T])
print 'Image number ',Output.shape



# Write it into csv
Output.T.to_csv('/Users/Nevaeh/PythonCSCourse/XcodePython/FindCell2/Step3_Performance/prediction.csv',index=None)
print "Result has been writen into csv files !"


Loading ... 
Loading done! time:  6.046407938
Done!
Image number  (9, 131072)
Result has been writen into csv files !


In [18]:
Output.shape

(9, 131072)

In [19]:
Output = pd.DataFrame(Output.values[-1,:]).T

In [20]:
Output.shape

(1, 131072)

In [21]:
pd.DataFrame(np.concatenate([x_data_test.reshape([-1]),x_data_test.reshape([-1])])).T.shape

(1, 131072)